In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter

%matplotlib inline

## Let's load our dataset

The [`read_csv`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) method from pandas is our friend here, it keeps things nice and simple, it can be configurable for lot's of options, but the default settings work for us here. The one argument we specify is the URL where the file is hosted. You could visit this URL, download the CSV, and explore it in Excel if you want.

This reads the csv provided, and converts it into a [`pd.DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) object.

In [2]:
df = pd.read_csv('https://storage.googleapis.com/pick-n-mix-1/convertcsv.csv')

print(df.info())

df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32844 entries, 0 to 32843
Data columns (total 6 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   LSOA code (2011)                            32844 non-null  object
 1   LSOA name (2011)                            32844 non-null  object
 2   Local Authority District code (2019)        32844 non-null  object
 3   Local Authority District name (2019)        32844 non-null  object
 4   Index of Multiple Deprivation (IMD) Rank    32844 non-null  int64 
 5   Index of Multiple Deprivation (IMD) Decile  32844 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 1.5+ MB
None


,LSOA code (2011),LSOA name (2011),Local Authority District code (2019),Local Authority District name (2019),Index of Multiple Deprivation (IMD) Rank,Index of Multiple Deprivation (IMD) Decile
0,E01000001,City of London 001A,E09000001,City of London,29199,9
1,E01000002,City of London 001B,E09000001,City of London,30379,10
2,E01000003,City of London 001C,E09000001,City of London,14915,5
3,E01000005,City of London 001E,E09000001,City of London,8678,3
4,E01000006,Barking and Dagenham 016A,E09000002,Barking and Dagenham,14486,5


## Validation

Let's make sure that the data meets some of our expectations. The `Index of Multiple Deprivation (IMD) Rank` column is a ranking, therefore no two LSOA locations should have the same ranking. Let's check if that assumption holds true?

We can count the number of times each value occurs in a column with the [`.value_counts`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html) method. One important thing to note here, you might notice, when reading the documentation, this method is used from a [`pd.Series`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html) object, rather than a [`pd.DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) object.

Basically, each DataFrame is made up of Series objects. Each Series represents a column of data. A DataFrame is a collection of one or more columns.

In [3]:
df['Index of Multiple Deprivation (IMD) Rank'].value_counts()

17970    2
26064    2
6891     2
16434    2
25064    2
        ..
8881     1
15026    1
12979    1
2740     1
2049     1
Name: Index of Multiple Deprivation (IMD) Rank, Length: 32818, dtype: int64

We can see that some of these ranks occur more than once. We should check how often we're seeing ranks that occur more than once. Let's use a [`Counter`](https://docs.python.org/3.7/library/collections.html#collections.Counter) from the collections package, which comes with Python, to count the number of occurences of each rank.

In [4]:
Counter(df['Index of Multiple Deprivation (IMD) Rank'].value_counts())

Counter({2: 26, 1: 32792})

So what's this telling us?

This Counter instance shows that there are 26 ranks that occur twice, but 32792 ranks that occur once. So presumably two different LSOA are tied as the 17970th ranked IMD.

Our assumption was proved incorrect, but we were close, and now we have a better understanding of the dataset.

## Filtering

Filtering is an important tool when we're exploring and analysing data.

My preferred approach to this is using the [`Boolean indexing`](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#boolean-indexing) method.

Confusingly [`pd.filter`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.filter.html) subsets the data according to columns or rows specified, rather than by values.

Let's look for Manchester LSOAs.

In [5]:
df[(df['Local Authority District name (2019)'] == 'Manchester')]

,LSOA code (2011),LSOA name (2011),Local Authority District code (2019),Local Authority District name (2019),Index of Multiple Deprivation (IMD) Rank,Index of Multiple Deprivation (IMD) Decile
4935,E01005061,Manchester 018A,E08000003,Manchester,724,1
4936,E01005062,Manchester 018B,E08000003,Manchester,11725,4
4937,E01005063,Manchester 018C,E08000003,Manchester,1101,1
4938,E01005065,Manchester 018D,E08000003,Manchester,1994,1
4939,E01005066,Manchester 018E,E08000003,Manchester,5152,2
...,...,...,...,...,...,...
32759,E01033684,Manchester 013F,E08000003,Manchester,16686,6
32760,E01033685,Manchester 003F,E08000003,Manchester,11316,4
32761,E01033686,Manchester 028F,E08000003,Manchester,5347,2
32762,E01033687,Manchester 003G,E08000003,Manchester,3067,1


Let's go one step further, let's look at only LSOAs in Manchester, but also with an IMD decile greater than 8.

In [6]:
df[(df['Local Authority District name (2019)'] == 'Manchester') &
   (df['Index of Multiple Deprivation (IMD) Decile'] >= 8)]

,LSOA code (2011),LSOA name (2011),Local Authority District code (2019),Local Authority District name (2019),Index of Multiple Deprivation (IMD) Rank,Index of Multiple Deprivation (IMD) Decile
5017,E01005155,Manchester 037C,E08000003,Manchester,25023,8
5019,E01005157,Manchester 037D,E08000003,Manchester,26894,9
5031,E01005169,Manchester 045C,E08000003,Manchester,24162,8
5036,E01005174,Manchester 043C,E08000003,Manchester,25466,8
5137,E01005277,Manchester 038D,E08000003,Manchester,25774,8


## Simple Transformation

Let's convert the Local Authority District name into `ALL CAPS`.

In [12]:
df['Local Authority District name (2019)'] = df['Local Authority District name (2019)'].apply(str.upper)

df.head()

,LSOA code (2011),LSOA name (2011),Local Authority District code (2019),Local Authority District name (2019),Index of Multiple Deprivation (IMD) Rank,Index of Multiple Deprivation (IMD) Decile
0,E01000001,City of London 001A,E09000001,CITY OF LONDON,29199,9
1,E01000002,City of London 001B,E09000001,CITY OF LONDON,30379,10
2,E01000003,City of London 001C,E09000001,CITY OF LONDON,14915,5
3,E01000005,City of London 001E,E09000001,CITY OF LONDON,8678,3
4,E01000006,Barking and Dagenham 016A,E09000002,BARKING AND DAGENHAM,14486,5


## Split Apply Combine

This is a really important concept, expanded upon by Hadley Wickham [here](https://www.jstatsoft.org/article/view/v040i01).

The core idea is that we `split` a big dataframe into small groups, `apply` a function, or set of functions on each group, and then `combine` the results together by some form of aggregation.

Let's get the mean IMD rank for each LSOA.

In [23]:
df.groupby('Local Authority District code (2019)') \
    .agg({'Index of Multiple Deprivation (IMD) Rank': 'mean'})

,Index of Multiple Deprivation (IMD) Rank
Local Authority District code (2019),
E06000001,10395.965517
E06000002,9189.569767
E06000003,12470.977273
E06000004,15477.158333
E06000005,14055.938462
...,...
E09000029,21630.719008
E09000030,10369.312500
E09000031,11735.375000


How does it actually work under the hood?

In [18]:
for name, group in df.groupby('Local Authority District code (2019)'):
    break
    
print(name)

group

E06000001


,LSOA code (2011),LSOA name (2011),Local Authority District code (2019),Local Authority District name (2019),Index of Multiple Deprivation (IMD) Rank,Index of Multiple Deprivation (IMD) Decile
11596,E01011949,Hartlepool 009A,E06000001,HARTLEPOOL,4731,2
11597,E01011950,Hartlepool 008A,E06000001,HARTLEPOOL,527,1
11598,E01011951,Hartlepool 007A,E06000001,HARTLEPOOL,1663,1
11599,E01011952,Hartlepool 002A,E06000001,HARTLEPOOL,3469,2
11600,E01011953,Hartlepool 002B,E06000001,HARTLEPOOL,911,1
11601,E01011954,Hartlepool 001A,E06000001,HARTLEPOOL,5729,2
11602,E01011955,Hartlepool 003A,E06000001,HARTLEPOOL,848,1
11603,E01011957,Hartlepool 003C,E06000001,HARTLEPOOL,4716,2
11604,E01011959,Hartlepool 014A,E06000001,HARTLEPOOL,22123,7
11605,E01011960,Hartlepool 014B,E06000001,HARTLEPOOL,23195,8
